In [11]:
import numpy as np
from my_stuff import MufexKeys
from quantfreedom.enums import *
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.simulate import run_df_backtest, or_backtest
from datetime import datetime

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
import plotly.graph_objects as go

from logging import getLogger
from typing import NamedTuple

from quantfreedom.helper_funcs import cart_product
from quantfreedom.indicators.tv_indicators import sma_tv
from quantfreedom.enums import CandleBodyType
from quantfreedom.strategies.strategy import Strategy

logger = getLogger("info")


class IndicatorSettingsArrays(NamedTuple):
    sma_fast_length: np.array
    sma_slow_length: np.array


class SMACrossing(Strategy):
    def __init__(
        self,
        long_short: str,
        sma_fast_length: np.array,
        sma_slow_length: np.array,
    ) -> None:
        self.long_short = long_short

        cart_arrays = cart_product(
            IndicatorSettingsArrays(
                sma_fast_length=sma_fast_length,
                sma_slow_length=sma_slow_length,
            )
        )

        sma_fast_length = cart_arrays[0].astype(np.int_)
        sma_slow_length = cart_arrays[1].astype(np.int_)

        sma_bools = sma_fast_length < sma_slow_length

        self.indicator_settings_arrays: IndicatorSettingsArrays = IndicatorSettingsArrays(
            sma_fast_length=sma_fast_length[sma_bools],
            sma_slow_length=sma_slow_length[sma_bools],
        )

        if long_short == "long":
            self.set_entries_exits_array = self.long_set_entries_exits_array
            self.log_indicator_settings = self.long_log_indicator_settings
            self.entry_message = self.long_entry_message
        else:
            self.set_entries_exits_array = self.short_set_entries_exits_array
            self.log_indicator_settings = self.short_log_indicator_settings
            self.entry_message = self.short_entry_message

    #######################################################
    #######################################################
    #######################################################
    ##################      Long     ######################
    ##################      Long     ######################
    ##################      Long     ######################
    #######################################################
    #######################################################
    #######################################################

    def long_set_entries_exits_array(
        self,
        candles: np.array,
        ind_set_index: int,
    ):
        try:
            closes = candles[:, CandleBodyType.Close]

            # sma fast
            self.sma_fast_length = self.indicator_settings_arrays.sma_fast_length[ind_set_index]
            sma_fast = sma_tv(
                source=closes,
                length=self.sma_fast_length,
            )

            self.sma_fast = np.around(sma_fast, 2)

            self.prev_sma_fast = np.roll(self.sma_fast, 1)
            self.prev_sma_fast[0] = np.nan
            logger.info(f"Created sma_fast sma_fast_length= {self.sma_fast_length}")

            # sma slow
            self.sma_slow_length = self.indicator_settings_arrays.sma_slow_length[ind_set_index]
            sma_slow = sma_tv(
                source=closes,
                length=self.sma_slow_length,
            )

            self.sma_slow = np.around(sma_slow, 2)

            self.prev_sma_slow = np.roll(self.sma_slow, 1)
            self.prev_sma_slow[0] = np.nan
            logger.info(f"Created sma_slow sma_slow_length= {self.sma_slow_length}")

            # Entries
            self.entries = (self.prev_sma_fast < self.prev_sma_slow) & (self.sma_fast > self.sma_slow)
            self.cross_above_signal = np.where(self.entries, sma_fast, np.nan)
            logger.debug("Created entries and cross above signals")

            # Exits
            exits = (self.prev_sma_fast > self.prev_sma_slow) & (self.sma_fast < self.sma_slow)
            self.cross_below_signal = np.where(exits, sma_fast, np.nan)
            self.exit_prices = np.roll(np.where(exits, closes, np.nan), 1)
            logger.debug("Created exit prices and cross below signals")

        except Exception as e:
            logger.error(f"Exception long_set_entries_exits_array -> {e}")
            raise Exception(f"Exception long_set_entries_exits_array -> {e}")

    def long_log_indicator_settings(
        self,
        ind_set_index: int,
    ):
        logger.info(
            f"Indicator Settings Index= {ind_set_index}\
            \nsma_fast_length= {self.sma_fast_length}\
            \nsma_slow_length= {self.sma_slow_length}"
        )

    def long_entry_message(
        self,
        bar_index: int,
    ):
        logger.info("\n\n")
        logger.info("Entry time!!! prev_sma_fast < prev_sma_slow & sma_fast > sma_slow")
        logger.info(
            f"{self.prev_sma_fast[bar_index]} < {self.prev_sma_slow[bar_index]} & {self.sma_fast[bar_index]} > {self.sma_slow[bar_index]}"
        )

    #######################################################
    #######################################################
    #######################################################
    ##################      short    ######################
    ##################      short    ######################
    ##################      short    ######################
    #######################################################
    #######################################################
    #######################################################

    def short_set_entries_exits_array(
        self,
        candles: np.array,
        ind_set_index: int,
    ):
        pass

    def short_log_indicator_settings(
        self,
        ind_set_index: int,
    ):
        pass

    def short_entry_message(
        self,
        bar_index: int,
    ):
        pass

    #######################################################
    #######################################################
    #######################################################
    ##################      Live     ######################
    ##################      Live     ######################
    ##################      Live     ######################
    #######################################################
    #######################################################
    #######################################################

    def live_set_indicator(
        self,
        closes: np.array,
    ):
        pass

    def live_evaluate(
        self,
        candles: np.array,
    ):
        pass

    #######################################################
    #######################################################
    #######################################################
    ##################      Plot     ######################
    ##################      Plot     ######################
    ##################      Plot     ######################
    #######################################################
    #######################################################
    #######################################################

    def plot_signals(
        self,
        candles: np.array,
    ):
        timestamp = candles[:, CandleBodyType.Timestamp]
        datetimes = timestamp.astype("datetime64[ms]")
        open = candles[:, CandleBodyType.Open]
        high = candles[:, CandleBodyType.High]
        low = candles[:, CandleBodyType.Low]
        close = candles[:, CandleBodyType.Close]

        fig = go.Figure(
            data=[
                go.Candlestick(
                    x=datetimes,
                    open=open,
                    high=high,
                    low=low,
                    close=close,
                    name="Candles",
                ),
                go.Scatter(
                    x=datetimes,
                    y=self.sma_fast,
                    name="SMA Fast",
                    line_color="lightblue",
                ),
                go.Scatter(
                    x=datetimes,
                    y=self.sma_slow,
                    name="SMA Slow",
                    line_color="yellow",
                ),
                go.Scatter(
                    x=datetimes,
                    y=self.cross_above_signal,
                    mode="markers",
                    name="Entries",
                    marker=dict(
                        size=12,
                        symbol="circle",
                        color="#00F6FF",
                        line=dict(
                            width=1,
                            color="DarkSlateGrey",
                        ),
                    ),
                ),
                go.Scatter(
                    x=datetimes,
                    y=self.cross_below_signal,
                    mode="markers",
                    name="Exits",
                    marker=dict(
                        size=12,
                        symbol="triangle-up",
                        color="#FF7B00",
                        line=dict(
                            width=1,
                            color="DarkSlateGrey",
                        ),
                    ),
                ),
            ]
        )
        fig.update_layout(
            height=800,
            xaxis_rangeslider_visible=False,
            title=dict(
                x=0.5,
                text="Signals",
                xanchor="center",
                font=dict(
                    size=40,
                ),
            ),
        )
        fig.show()

    def get_strategy_plot_filename(
        self,
        candles: np.array,
    ):
        pass

In [13]:
mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

In [14]:
candles = mufex_main.get_candles(
    symbol="BTCUSDT",
    timeframe="30m",
    candles_to_dl=3000,
    # since_datetime=datetime(2023, 11, 14, 19),
    # until_datetime=datetime(2023, 11, 16),
)

In [15]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)

backtest_settings = BacktestSettings()
dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([0]),
    sl_based_on_add_pct=np.array([0.05, 0.15]),
    sl_based_on_lookback=np.array([30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([1, 2]),
    trail_sl_when_pct=np.array([1, 3]),
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="market",
    tp_strategy_type=TakeProfitStrategyType.Provided,
    trail_sl_bool=True,
    z_or_e_type=None,
)

strategy = SMACrossing(
    long_short="long",
    sma_fast_length=np.arange(20, 61, 20),
    sma_slow_length=np.arange(30, 91, 30),
)

In [16]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=mufex_main.exchange_settings,
    static_os=static_os,
    strategy=strategy,
    logger_bool=False,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 8
Total combinations of settings to test: 48
Total candles: 3,000
Total candles to test: 144,000


In [18]:
backtest_results.sort_values(by=["qf_score"], ascending=False)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
2,0,2,24.0,9,15,434.199,37.500,0.893,917.707,4341.992,5341.992
4,0,4,38.0,20,18,212.743,52.632,0.871,472.895,2127.428,3127.428
0,0,0,42.0,20,22,193.558,47.619,0.867,492.568,1935.580,2935.580
6,0,6,23.0,9,14,336.110,39.130,0.809,827.190,3361.105,4361.105
21,2,5,9.0,7,2,181.346,77.778,0.806,53.044,1813.460,2813.460
17,2,1,9.0,7,2,207.256,77.778,0.802,60.463,2072.559,3072.559
1,0,1,30.0,11,19,332.648,36.667,0.784,971.626,3326.475,4326.475
16,2,0,15.0,12,3,65.298,80.000,0.771,41.399,652.977,1652.977
20,2,4,15.0,12,3,57.328,80.000,0.770,37.151,573.281,1573.281
3,0,3,24.0,8,16,294.354,33.333,0.727,795.938,2943.543,3943.543
